In [6]:
import pdfplumber
import pandas as pd
import re

def is_nutrient_value(val):
    """
    Checks if a string looks like a nutrient value.
    Valid formats: '123', '12,3', 'Tr', 'NA', '*', empty string.
    Invalid: 'Arroz', 'Cozido', long text.
    """
    if not val: return True # Empty cells are common in data
    val = str(val).strip().replace('\n', ' ')
    if val in ['NA', 'Tr', '*', '']: return True
    # Check for numbers (allow comma or dot)
    if re.match(r'^[\d,.]+$', val): return True
    return False

def extract_taco_robust(pdf_path, output_csv):
    data_store = {} # Key: ID, Value: {name, macros: [], micros: []}
    
    # Page Ranges (Based on TACO 4th Ed)
    # Tables typically start on pg 29 (index 28) and end around pg 67 (index 66)
    start_page = 28
    end_page = 68

    print(f"Scanning {pdf_path} from page {start_page + 1} to {end_page}...")

    with pdfplumber.open(pdf_path) as pdf:
        for i in range(start_page, end_page):
            page = pdf.pages[i]
            text = page.extract_text() or ""
            
            # --- Identify Page Type ---
            page_type = None
            if "Umidade" in text and "Proteína" in text:
                page_type = "MACRO"
            elif "Manganês" in text and "Fósforo" in text:
                page_type = "MICRO"
            else:
                continue

            # Extract table using "text" strategy (best for grabbing all words)
            # We assume stream logic because lines are sometimes missing in PDF metadata
            table = page.extract_table(table_settings={
                "vertical_strategy": "text", 
                "horizontal_strategy": "text",
                "intersection_tolerance": 10
            })

            if not table: continue

            for row in table:
                # Clean basic whitespace
                row = [str(cell).replace('\n', ' ').strip() if cell else "" for cell in row]
                
                # Filter noise rows
                if not row or len(row) < 5: continue
                
                # Find ID (First numeric looking item in the first 2 slots)
                food_id = None
                id_idx = -1
                
                if row[0].isdigit():
                    food_id = row[0]
                    id_idx = 0
                elif len(row) > 1 and row[1].isdigit():
                    food_id = row[1]
                    id_idx = 1
                
                if not food_id: continue # Header or category row

                # Ensure ID exists in store
                if food_id not in data_store:
                    data_store[food_id] = {'id': food_id, 'name': '', 'macros': [], 'micros': []}

                # --- RIGHT-TO-LEFT PARSING ---
                if page_type == "MACRO":
                    # Expected standard columns after name: 
                    # [Umidade, kcal, kJ, Prot, Lip, Chol, Carb, Fiber, Ash, Ca, Mg] = 11 cols
                    # Sometimes kcal/kJ are merged -> 10 cols
                    
                    data_chunk = []
                    # Try taking last 11
                    if len(row) >= 11 + (id_idx + 1):
                        potential_data = row[-11:]
                        # Check validity of this chunk
                        if sum(is_nutrient_value(x) for x in potential_data) >= 8: # Tolerance for some weird OCR
                            data_chunk = potential_data
                            name_chunk = row[id_idx+1 : -11]
                        else:
                            # Try 10 columns (Energy merged)
                            potential_data = row[-10:]
                            if sum(is_nutrient_value(x) for x in potential_data) >= 7:
                                # Split the merged Energy column (Col 1 in this chunk)
                                # value "517 124" -> "517", "124"
                                energy = potential_data[1].split(' ')
                                if len(energy) == 2:
                                    data_chunk = [potential_data[0]] + energy + potential_data[2:]
                                else:
                                    # Fallback if split fails
                                    data_chunk = [potential_data[0], potential_data[1], ""] + potential_data[2:]
                                name_chunk = row[id_idx+1 : -10]
                    
                    if data_chunk:
                        # Reconstruct Name
                        full_name = " ".join(name_chunk).strip()
                        # Fix common PDF join issues
                        full_name = full_name.replace(" ,", ",").replace("  ", " ")
                        
                        if len(full_name) > 2:
                            data_store[food_id]['name'] = full_name
                        data_store[food_id]['macros'] = data_chunk

                elif page_type == "MICRO":
                    # Expected columns after ID:
                    # [Mn, P, Fe, Na, K, Cu, Zn, Retinol, RE, RAE, B1, B2, B6, Niacin, VitC] = 15 cols
                    target_cols = 15
                    
                    if len(row) >= target_cols + 1: # +1 for ID
                        # Take last 15
                        data_chunk = row[-target_cols:]
                        # Simple validation
                        if sum(is_nutrient_value(x) for x in data_chunk) >= 10:
                            data_store[food_id]['micros'] = data_chunk

    # --- Formatting for Output ---
    print("Formatting data...")
    final_rows = []
    
    headers_macro = ["Umidade(%)", "Energia(kcal)", "Energia(kJ)", "Proteína(g)", "Lipídeos(g)", 
                     "Colesterol(mg)", "Carboidrato(g)", "Fibra(g)", "Cinzas(g)", "Cálcio(mg)", "Magnésio(mg)"]
    
    headers_micro = ["Manganês(mg)", "Fósforo(mg)", "Ferro(mg)", "Sódio(mg)", "Potássio(mg)", 
                     "Cobre(mg)", "Zinco(mg)", "Retinol(mcg)", "RE(mcg)", "RAE(mcg)", 
                     "Tiamina(mg)", "Riboflavina(mg)", "Piridoxina(mg)", "Niacina(mg)", "VitaminaC(mg)"]

    all_headers = ["ID", "Alimento"] + headers_macro + headers_micro

    # Sort keys numerically
    sorted_ids = sorted(data_store.keys(), key=lambda x: int(x) if x.isdigit() else 9999)

    for fid in sorted_ids:
        item = data_store[fid]
        
        # Pad macros if missing (rare, but safety check)
        m_list = item['macros']
        if len(m_list) < 11: m_list += [""] * (11 - len(m_list))
        if len(m_list) > 11: m_list = m_list[:11] # Trim if extraction was weird
        
        # Pad micros
        mi_list = item['micros']
        if len(mi_list) < 15: mi_list += [""] * (15 - len(mi_list))
        
        full_row = [item['id'], item['name']] + m_list + mi_list
        final_rows.append(full_row)

    # Convert to DF
    df = pd.DataFrame(final_rows, columns=all_headers)
    
    # Clean numeric format (Brazil , to Int'l .)
    for col in all_headers[2:]:
        df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
        # Remove any internal spaces in numbers e.g. "1 000" -> "1000"
        df[col] = df[col].str.replace(' ', '', regex=False)

    df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"Extraction Complete! {len(df)} items saved to {output_csv}")

# --- Run ---
extract_taco_robust("taco_4_edicao_ampliada_e_revisada.pdf", "taco_final.csv")

Scanning taco_4_edicao_ampliada_e_revisada.pdf from page 29 to 68...
Formatting data...
Extraction Complete! 597 items saved to taco_final.csv
